# Use Case Load Grand Ensemble output fast

Challenge:
    - mm output in 1 file per year
    - concat over time, and member takes very long
    
Solution:
    - somehow speedup concat over time and member
    - postprocess (optional with cdo) and save in one file, then load this one

# Solution: postprocess and save in one file

In [2]:
#PMMPIESM implementation

# Solution: speedup concat

In [3]:
# decode_times=False, decode_cf=False